Score a candidate ms2 peak according to its coverage of the feature's drift and RT

In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import peakutils
import json

In [3]:
db_name = '/home/ubuntu/UPS2_allion/UPS2_allion-features-1-1097.sqlite'
feature_db_name = '/home/ubuntu/UPS2_allion/UPS2_allion.sqlite'
feature_id = 23
ms2_peak_id = 2

In [4]:
# load the summary information about all the peaks for this feature
db_conn = sqlite3.connect(db_name)
ms2_peaks_df = pd.read_sql_query("select * from ms2_peaks where feature_id={} order by peak_id ASC".format(feature_id), db_conn)
db_conn.close()

In [15]:
ms2_peaks_df.head()

,feature_id,peak_id,centroid_mz,composite_mzs,centroid_scan,intensity
0,23,1,588.361,"[588337, 588338, 588339, 588340, 588341, 58834...",107,8997446
1,23,2,861.007,"[860972, 860973, 860974, 860975, 860976, 86097...",108,10596077
2,23,3,861.508,"[861471, 861472, 861473, 861474, 861475, 86147...",108,10320728
3,23,4,775.952,"[775922, 775923, 775924, 775925, 775926, 77592...",108,8249447
4,23,5,776.454,"[776422, 776423, 776424, 776425, 776426, 77642...",108,7360819


In [5]:
# get the raw points for this feature (note we don't have the raw points assigned to peaks in this table)
db_conn = sqlite3.connect(db_name)
ms2_feature_region_raw_points_df = pd.read_sql_query("select frame_id,scan,scaled_mz from ms2_feature_region_points where feature_id={}".format(feature_id), db_conn)
db_conn.close()

In [6]:
# calculate the feature's available "slots" in drift and RT
db_conn = sqlite3.connect(feature_db_name)
feature_df = pd.read_sql_query("select * from features where feature_id={}".format(feature_id), db_conn)
db_conn.close()

In [7]:
feature_df

,feature_id,base_frame_id,base_cluster_id,charge_state,start_frame,end_frame,quality_score,summed_intensity,scan_lower,scan_upper,mz_lower,mz_upper
0,23,5755,1,3,5748,5761,1.0,542463190,98,118,641.041565,643.049683


In [12]:
feature_slots = (feature_df.end_frame-feature_df.start_frame).item()*(feature_df.scan_upper-feature_df.scan_lower).item()
feature_slots

260

In [14]:
# work out how many of the slots are filled by each peak's raw points
# for ms2_peak_idx in range(len(ms2_peaks_df)):
ms2_peak_idx = 1
ms2_peak_id = ms2_peaks_df.loc[ms2_peak_idx].peak_id.astype(int)
composite_mzs_min = ms2_peaks_df.loc[ms2_peak_idx].composite_mzs_min.item()
composite_mzs_max = ms2_peaks_df.loc[ms2_peak_idx].composite_mzs_max.item()
peak_raw_points_df = ms2_feature_region_raw_points_df[(ms2_feature_region_raw_points_df.scaled_mz >= composite_mzs_min) & (ms2_feature_region_raw_points_df.scaled_mz <= composite_mzs_max)].copy()
# drop the duplicate rows
peak_raw_points_df.drop_duplicates(subset=('scan','frame_id'), inplace=True)


AttributeError: 'Series' object has no attribute 'composite_mzs_min'